# That last one

We are not gonna do a lot of new things today. Mostly we'll just combine what you have learned in order to make our final prediction. It is important that you have successfully run week12split.ipynb (in geo_env) before starting on this exercises (in xgb_env). The reason is that we a going to use the pickles generated in week12split.ipynb Lets get to it.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import pickle
import time

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

# 5B.1 Getting Started
Load y_train_week12.pkl, X_train_week12.pkl, y_val_week12.pkl, X_val_week12.pkl which you created in week12split.ipynb

In [ ]:
pkl_file = ...
y_train = ...
pkl_file.close()

pkl_file = ...
X_train = ...
pkl_file.close()

pkl_file = ...
y_val = ...
pkl_file.close()

pkl_file = ...
X_val = ...
pkl_file.close()

Do what ever you need to make sure the data is fine

Now that we have included xcoord and ycoord we can plot our plots a different way than usual. It is slightly more involved, but way faster and way more flexible. What you see below is simply a scatter plot where x is xcoord and y is ycoord. The scatters are then colored (c) in regards to some feature. I used 'log_best_decay10' but fell free to experiment. Note that we need to specific year for each input or the scatter plot will end up containing all years. I've sat makers to 's' (squares) so the scatter points mimics the pixels we usually plot.

In [ ]:
plt.figure(figsize = [22,10])
plt.scatter(X_train[X_train['year'] == 2012]['xcoord'], X_train[X_train['year'] == 2012]['ycoord'], c = X_train[X_train['year'] == 2012]['log_best_decay10'], s=1, marker='s')
plt.show()

# 5B.2 Feature Selection

We looked at feature selection before. Here we are gonna choose a subset of features make sure we can save some training time (there is gonna be a lot of that below) while not compromising our predictive capabilities. We are gonna use an XGB model for this purpose.  

Start by defining and fitting a XGBClassifier on X_train and y_train\['binary_best_t1'\]. Set number of estimators to 16. This is very low, but you can always go back and set it higher once you have the whole exercise running. Now create and print your training predictions and validation predictions.

In [ ]:
model = ... # 16 estimators is very low. Try 32, 64 or 128 if you have the time
model.fit(..., ...)

In [ ]:
y_train_pred = ...
y_val_pred = ...

In [ ]:
AUC_train = ...
AP_train = ...
BS_train = ...

AUC_val = ...
AP_val = ...
BS_val = ...

print(f'\tTrain\tVal\nAUC: \t{AUC_train:.3f}\t{AUC_val:.3f}\nAP: \t{AP_train:.3f}\t{AP_val:.3f}\nBS: \t{BS_train:.3f}\t{BS_val:.3f}')

Now, you of course remember how we created the importance plots for Random Forest and XGboost. We can use such importance measures the rank our features choosing the best ones. To know how many to included we can start be including the most important, then the next most important and so forth. We stop when our predictive capabilities does not get better of if they start to get worse. The code below prints out the 15 most important features given the model we just fitted. They are ordered so the last on is the most important, the second to last the second most important one and so on.

In [ ]:
sorted_idx = model.feature_importances_.argsort()
X_train.columns[sorted_idx][-15:]

This here loop will iterate over the 20 most important features and create increasingly large models (could be all features; 20 is to save time). We save (and print) the evaluation metrics as the model gets bigger. No code needs changing but make sure you understand what is going on.

In [ ]:
start_time = time.time()

AUC_train_list = []
AP_train_list = []
BS_train_list = []

AUC_val_list = []
AP_val_list = []
BS_val_list = []

for i in np.arange(1,21,1):
    
    selected_features = X_train.columns[sorted_idx][-i:]
    X_train_slim = X_train[selected_features]
    X_val_slim = X_val[selected_features]
    
    model_tmp = XGBClassifier(n_estimators=16, random_state = 42)# 16 estimators is very low. Try 32, 64 or 128 if you have the time
    model_tmp.fit(X_train_slim, y_train['binary_best_t1'])
    
    y_train_pred = model_tmp.predict_proba(X_train_slim)[:,1]
    y_val_pred = model_tmp.predict_proba(X_val_slim)[:,1]
    
    AUC_train_list.append(metrics.roc_auc_score(y_train['binary_best_t1'], y_train_pred))
    AP_train_list.append(metrics.average_precision_score(y_train['binary_best_t1'], y_train_pred))
    BS_train_list.append(metrics.brier_score_loss(y_train['binary_best_t1'], y_train_pred))

    AUC_val_list.append(metrics.roc_auc_score(y_val['binary_best_t1'], y_val_pred))
    AP_val_list.append(metrics.average_precision_score(y_val['binary_best_t1'], y_val_pred))
    BS_val_list.append(metrics.brier_score_loss(y_val['binary_best_t1'], y_val_pred))
    
    print(f'{i}/15 AP (val): {AP_val_list[i-1]}, using {selected_features}\n')
    

end_time = time.time()
run_time = (end_time - start_time)/60
print(f'minutes to run: {run_time:.2}')

From the printout above you should have a pretty good idea of when the model performance saturates. Lets make this even more explicit by plotting how the evaluation metrics evolved as functions of included features. No code needs changing, but feel free the optimize and prettify

In [ ]:
grid = np.arange(1,21,1)

plt.title('AUC')
plt.plot(grid, AUC_train_list, '--', label = 'train')
plt.plot(grid, AUC_val_list,  '-', label = 'val')
plt.legend()
plt.show()

plt.title('AP')
plt.plot(grid, AP_train_list, '--', label = 'train')
plt.plot(grid, AP_val_list, '-', label = 'val')

plt.legend()
plt.show()

plt.title('BS')
plt.plot(grid, BS_train_list, '--', label = 'train')
plt.plot(grid, BS_val_list, '-', label = 'val')
plt.legend()
plt.show()


Now, lets slim down our X_train and X_val to X_train_slim and X_val_slim. Simply choose a subset of features given the results above. You can choose the number manually or just take the 'argmax' to get the best number given some metric. No code needs changing, but as it is now we take the number which optimized the AP (val).

In [ ]:
optim = np.argmax(np.array(AP_val)) # you can take the number that optimizes some metric, here AP.

# optim = 5 # you can set it by hand


selected_features = X_train.columns[sorted_idx][-optim:]

print(selected_features)

X_train_slim = X_train[selected_features]
X_val_slim = X_val[selected_features]



Test out your "slim" data by defining, fitting and evaluating an XGB classifier to y_train\['binary_best_t1'\] - is it as good as a model using all features?

From here on - until 5B.5 - when I say fit a model i mean fit a model to X_train_slim and y_train\['binary_best_t1'\] unless something else is explicitly stated



# 5B.3 Grid Search (again)

Here we are going to do grid search. This is gonna be pretty much like last time except: we are only gonna use full grid search and beyond XGboot your are also gonna apply grid search on a KNeighbors classifier, a naive Bayes classifier (BernoulliNB), an AdaBoost classifier and a Random Forest classifier. I have proposed which hyper parameters and ranges you survey, but feel free to experiment. The first one is a freebie so no code needs to be changed here:

In [ ]:
folds = 2 # 2 folds is very low, try 3, 6 or 9 if you have the time
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)

In [ ]:
XGBparams = {
        'max_depth': [4, 5],
        'learning_rate': [0.1, 0.5],
        'gamma': [0.1, 1],
        'reg_lambda': [0.01, 0.1]
        }

XGBmodel = XGBClassifier(n_estimators=16, random_state = 42)# 16 estimators is very low. Try 32, 64 or 128 if you have the time

XGBgrid = GridSearchCV(estimator=XGBmodel, param_grid=XGBparams, scoring='average_precision', n_jobs=-1, cv=skf.split(X_train_slim, y_train['binary_best_t1']), verbose=3 )

start_time = time.time()
XGBgrid.fit(X_train,y_train['binary_best_t1'])
end_time = time.time()

run_time = (end_time - start_time)/60

print(f'minutes to run: {run_time:.2}')

Now we use the new fund parameters to define a model. Don't fit or evaluate it just yet. We'll use it soon enough.

In [ ]:
XGBmodel_final = XGBClassifier(n_estimators = 16,  # 16 estimators is very low. Try 32, 64 or 128 if you have the time
                       reg_lambda = XGBgrid.best_params_['reg_lambda'],
                       max_depth = XGBgrid.best_params_['max_depth'], 
                       learning_rate = XGBgrid.best_params_['learning_rate'],
                       gamma =  XGBgrid.best_params_['gamma'], 
                       random_state = 42)

print(XGBmodel_final.get_params())

Now, do a grid search on a KNeighborsClassifier. The procedure is the same as above and I have supplied a dictionary with parameter ranges to be tested.


In [ ]:
KNNparams = {
        'n_neighbors' : [1,2,4],
        'weights': ['uniform', 'distance']
        }

KNNmodel = KNeighborsClassifier()

KNNgrid = GridSearchCV(estimator=..., param_grid=..., scoring='average_precision', n_jobs=-1, cv=skf.split(..., ...), verbose=3 )

start_time = time.time()

KNNgrid.fit(..., ...)

end_time = time.time()

run_time = (end_time - start_time)/60

print(f'minutes to run: {run_time:.2}')

Again, we define a model using the new found parameters

In [ ]:
KNNmodel_final = KNeighborsClassifier(n_jobs = -1, 
                                      n_neighbors = KNNgrid.best_params_['n_neighbors'], 
                                      weights =  KNNgrid.best_params_['weights'])

print(KNNmodel_final.get_params())

Now we trun to the Naive Bayes classifier. Do a grid search

In [ ]:

NBparams = {
        'alpha' : [0.1, 0.2, 0.5, 1],
        'fit_prior': [True, False]
        }

NBmodel = BernoulliNB()

NBgrid = ...

start_time = time.time()

NBgrid.fit(... , ...)

end_time = time.time()

run_time = (end_time - start_time)/60

print(f'minutes to run: {run_time:.2}')


Define a model using the new parameters

In [ ]:
NBmodel_final = BernoulliNB(alpha = NBgrid.best_params_['alpha'],
                            fit_prior = NBgrid.best_params_['fit_prior'])

print(NBmodel_final.get_params())

Now ADAboost:

In [ ]:
ADAparams = {
        'base_estimator' : [DecisionTreeClassifier(max_depth=2), DecisionTreeClassifier(max_depth=3)],
        'learning_rate': [0.5, 1]
        }

ADAmodel = ...  # set n_estimators at 16. It is very low so try 32, 64 or 128 if you have the time

ADAgrid = ...

start_time = time.time()
ADAgrid.fit(..., ...)
end_time = time.time()

run_time = (end_time - start_time)/60

print(f'minutes to run: {run_time:.2}')


Define a model

In [ ]:
ADAmodel_final = AdaBoostClassifier(n_estimators=16, random_state = 42, # 16 estimators is very low. Try 32, 64 or 128 if you have the time 
                                    base_estimator = ADAgrid.best_params_['base_estimator'], 
                                    learning_rate = ADAgrid.best_params_['learning_rate'])

print(ADAmodel_final.get_params())

And lastly, Random Forest:

In [ ]:

RFparams = {
        'criterion' : ['gini', 'entropy'],
        'max_depth': [3, 4],
        'class_weight': [None, 'balanced']
        }


RFmodel = ...  # set n_estimators at 32. It is very low so try 64 or 128 later if you have the time

RFgrid = ...

start_time = time.time()
RFgrid.fit(X_train,y_train['binary_best_t1'])
end_time = time.time()

run_time = (end_time - start_time)/60

print(f'minutes to run: {run_time:.2}')

Define the model

In [ ]:
RFmodel_final = RandomForestClassifier(n_estimators=32, n_jobs = -1, random_state = 42,  # 32 estimators is very low. Try 64 or 128 if you have the time
                                 criterion = RFgrid.best_params_['criterion'], 
                                 max_depth = RFgrid.best_params_['max_depth'], 
                                 class_weight = RFgrid.best_params_['class_weight'])

print(RFmodel.get_params())

# 5B.4 Le grand ensemble

Now, above you saved a number of "final" models. We are gonna use those now the create a larger voting ensemble as you also did in weeek 6. You don't need to use any weighting scheme just the "final" models defined after each grid search above. After the voting classifier is defined fit it to X_train_slim and y_train\['binary_best_t1'\]. Evaluate the voting ensemble model and print out the relevant training and validation metrics.


In [ ]:
eclf = ...
eclf = eclf.fit(..., ...)

In [ ]:
y_train_pred = ...
y_val_pred = ...

In [ ]:
AUC_train_eclf = ...
AP_train_eclf = ...
BS_train_eclf =...

AUC_val_eclf = ...
AP_val_eclf = ...
BS_val_eclf = ...

print(f'\tTrain\tVal\nAUC: \t{AUC_train_eclf:.3f}\t{AUC_val_eclf:.3f}\nAP: \t{AP_train_eclf:.3f}\t{AP_val_eclf:.3f}\nBS: \t{BS_train_eclf:.3f}\t{BS_val_eclf:.3f}')

# 5b.4 Trend

Now we a gonna see how our model does further out into the future. So at y_t2, y_t3, y_t4 and y_t5. Note that, if time permitted, we would do all optimization steps again with respect to each y, since we can't be sure it does not change. Today, however, we assume not much would change.

The way we do this is similar to the feature selection for-loop step above. Instead of iterating over different Xs we just iterate over the different ys. No code needs to be change below, but feel free to optimize and experiment.

Note, that this might take a loooonge time to run..

In [ ]:
start_time = time.time()

AUC_train_list = []
AP_train_list = []
BS_train_list = []

AUC_val_list = []
AP_val_list = []
BS_val_list = []

for i,j in enumerate(y_train.columns):

    model_tmp = VotingClassifier(estimators=[('XGB', XGBmodel), ('NB', NBmodel), ('ADA', ADAmodel), ('RF', RFmodel), ('KNN', KNNmodel)], voting='soft', n_jobs = -1)
    model_tmp.fit(X_train_slim, y_train[j])
    
    y_train_pred = model_tmp.predict_proba(X_train_slim)[:,1]
    y_val_pred = model_tmp.predict_proba(X_val_slim)[:,1]
    
    AUC_train_list.append(metrics.roc_auc_score(y_train[j], y_train_pred))
    AP_train_list.append(metrics.average_precision_score(y_train[j], y_train_pred))
    BS_train_list.append(metrics.brier_score_loss(y_train[j], y_train_pred))

    AUC_val_list.append(metrics.roc_auc_score(y_val[j], y_val_pred))
    AP_val_list.append(metrics.average_precision_score(y_val[j], y_val_pred))
    BS_val_list.append(metrics.brier_score_loss(y_val[j], y_val_pred))
    
    print(f'{i+1}/5 AP (val): {AP_val_list[i]}, y: {j}\n')
    

end_time = time.time()
run_time = (end_time - start_time)/60
print(f'minutes to run: {run_time:.2}')

Lets plot our results:

In [ ]:
grid = np.arange(1,6,1)

plt.figure(figsize = [15,5])

plt.title('AUC')
plt.plot(grid, AUC_train_list, '--', label = 'train')
plt.plot(grid, AUC_val_list,  '-', label = 'val')
plt.xlabel('years into the future')
plt.ylabel('score')
plt.legend()
plt.show()

plt.figure(figsize = [15,5])

plt.title('AP')
plt.plot(grid, AP_train_list, '--', label = 'train')
plt.plot(grid, AP_val_list, '-', label = 'val')
plt.xlabel('years into the future')
plt.ylabel('score')
plt.legend()
plt.show()

plt.figure(figsize = [15,5])

plt.title('BS')
plt.plot(grid, BS_train_list, '--', label = 'train')
plt.plot(grid, BS_val_list, '-', label = 'val')
plt.xlabel('years into the future')
plt.ylabel('score')
plt.legend()
plt.show()


Try to add the baselines models (persistence and ratio) to the three plots above for some context - how are we doing?

# 5B.5 TEST TIME

Now do the exact same thing you did above (5B.4) but now using the X_train_ttime, y_train_ttime, y_test and X_test. Not that in an authentic setting the test set can only be used once. So if you have any fine tuning you should go and do it now before evaluation against the test set. The scores you get a probably a bit worse than above but hopefully it is not too bad.

And you done!